In [1]:
#pip install pytesseract opencv-python-headless
#pip install llama-cpp-python
#!brew install tesseract -- install tessarct in mac

In [1]:
import cv2
import pytesseract
from llama_cpp import Llama
import json

ModuleNotFoundError: No module named 'llama_cpp'

In [20]:
file_path = "/Users/sivaramakrishnan/Development/Donut/invoice2.png"
model_name = "google/gemma-2b-it"
model_file = "/Users/sivaramakrishnan/.cache/lm-studio/models/Mozilla/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf"

In [21]:
# Set the path to Tesseract executable (adjust this based on your installation)
pytesseract.pytesseract.tesseract_cmd = '/opt/homebrew/bin/tesseract'

# Read the image (replace 'photo.jpg' with your image file path)
img = cv2.imread(file_path)

# Configuration for Tesseract (you can adjust this as needed)
config = ('-l eng --oem 1 --psm 3')

# Extract text from the image
extracted_text = pytesseract.image_to_string(img, config=config)
extracted_text

'INVOICE\n\nHOME\nFURNISHINGS\n\nYour business name, 126 Industry Road, Auckland 1061, New Zealand\n\nBILL TO\nYour client Invoice No.: 2022004\n75 Hamlin Road Issue date: 4/30/2022\nAuckland 1060 Due date: 5/14/2022\nNew Zealand\nReference: 2022004\nDESCRIPTION QUANTITY UNIT PRICE ($) AMOUNT ($)\nSample service 1 365.00 365.00\nTOTAL (NZD): $365.00\n\nT\n\nDUE (NZD) $365.00\n\nIssued by, signature:\n\nHome Furniakings\n\nYour business name, 126 Industry Road, Auckland 1061, New Zealand Email: email@yourbusinessname.co.nz\n\n'

In [22]:
llm = Llama(model_path=model_file,
            n_gpu_layers=-1)

llama_model_loader: loaded meta data with 21 key-value pairs and 164 tensors from /Users/sivaramakrishnan/.cache/lm-studio/models/Mozilla/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-2b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                          gemma.block_count u32              = 18
llama_model_loader: - kv   4:                     gemma.embedding_length u32              = 2048
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32              = 16384
llama_model_loader: - kv   6:                 gemma.attent

In [23]:
instruction = "<INST> extract data in json format with the document type </INST>\n" + extracted_text
response = llm(instruction, max_tokens=10000)
rtext = response['choices'][0]['text']
rtext


llama_print_timings:        load time =    1304.76 ms
llama_print_timings:      sample time =     514.81 ms /   220 runs   (    2.34 ms per token,   427.34 tokens per second)
llama_print_timings: prompt eval time =    1304.11 ms /   216 tokens (    6.04 ms per token,   165.63 tokens per second)
llama_print_timings:        eval time =   14598.14 ms /   219 runs   (   66.66 ms per token,    15.00 tokens per second)
llama_print_timings:       total time =   17039.58 ms /   435 tokens


'Please note that this invoice is subject to change.\n\nThis is a summary of the invoice. For the full details, please refer to the attached PDF document.\n\n**JSON Output:**\n\n```json\n{\n  "document_type": "INVOICE",\n  "customer_address": {\n    "street_address": "126 Industry Road, Auckland 1061, New Zealand",\n    "city": "Auckland",\n    "postal_code": "1060"\n  },\n  "invoice_details": {\n    "bill_to": {\n      "name": "Your client",\n      "invoice_no": "2022004"\n    },\n    "due_date": "05/14/2022",\n    "description": "Sample service 1 365.00 365.00",\n    "total_amount": "365.00"\n  }\n}\n```'

In [24]:
json_text = rtext[rtext.find("{"):rtext.rfind("}")+1]
json_text

'{\n  "document_type": "INVOICE",\n  "customer_address": {\n    "street_address": "126 Industry Road, Auckland 1061, New Zealand",\n    "city": "Auckland",\n    "postal_code": "1060"\n  },\n  "invoice_details": {\n    "bill_to": {\n      "name": "Your client",\n      "invoice_no": "2022004"\n    },\n    "due_date": "05/14/2022",\n    "description": "Sample service 1 365.00 365.00",\n    "total_amount": "365.00"\n  }\n}'

In [25]:
json.loads(json_text)

{'document_type': 'INVOICE',
 'customer_address': {'street_address': '126 Industry Road, Auckland 1061, New Zealand',
  'city': 'Auckland',
  'postal_code': '1060'},
 'invoice_details': {'bill_to': {'name': 'Your client',
   'invoice_no': '2022004'},
  'due_date': '05/14/2022',
  'description': 'Sample service 1 365.00 365.00',
  'total_amount': '365.00'}}